# Lesson 2 : LangGraph Components

In [ ]:
from utils.helpers import set_api_keys_env
set_api_keys_env()

#### Components

1. **Prompts** : Prompt templates allow reusable templates for generating prompts.
2. **Tools** : Tools are functions that can be called by the model to perform specific
3. **Graphs** : Graphs are the core of LangGraph, allowing you to define complex workflows. 
   1. Cyclic Graphs
   2. Persistence
   3. Human in the loop


* LangGraph is an extension of LangChain that supports graphs
* Single and Multi-Agent flows are described and represented as graphs
* Allows for extremely controlled flows
* Built-in persistence allows for human-in-the-loop workflows


#### Concepts:

1. **Nodes**: Agents,tools (functions)
2. **Edges**: Connections between nodes, representing the flow of data or control.
3. **Conditional Edges**: Decisions.


#### Data/State

1. Agent State  is accessible to all parts of the graph.
2. It is local to the graph
3. Can be stored in a persistence layer


#### Simple
```python
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

```

#### Complex
```python
class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[typle[AgentAction, str]], operator.add]
```



In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import Dict
from langchain_core.runnables import Runnable


In [ ]:
tool = TavilySearchResults(max_results=3)
print(type(tool))
print(tool.name)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
When you need to look up information, always use the search tool and return the result, not just say you will look it up.
"""

# You are a research assistant.
# from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI

# llm = init_chat_model("openai:gpt-4.1")
model = ChatOpenAI(model="gpt-4-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)



In [ ]:
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [ ]:
messages = [HumanMessage(content="What is the weather in Banglore today?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result['messages'][-1].content


In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result['messages'][-1].content)

In [ ]:
query = "What is the latest update on AirIndia 171 flight crash?" 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)


In [ ]:
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result['messages'][-1].content)